# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import patsy

import statsmodels.api as sm
import statsmodels.formula.api as stm

from sklearn import tree
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
#Remover colunas que não serão usadas
colunas_drop = ['Unnamed: 0', 'data_ref', 'data_ref', 'id_cliente']
df.drop(colunas_drop, axis=1, inplace=True)

#Tratamento dados faltantes
df['tempo_emprego'].fillna(value = df['tempo_emprego'].median(), inplace=True)


df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

# 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [5]:
df_treino, df_teste = train_test_split(df, test_size=0.25, random_state=666) 

# 2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?

##### Criação do modelo

In [6]:
#Função para definir diferentes modelos

def criar_modelo(data_frame: pd.DataFrame, formula_modelo: str, l1: int, alph: float):
    
    modelo = stm.ols(data = df_treino,
                     formula = formula_modelo).fit_regularized(method = 'elastic_net', 
                                                               refit = True, 
                                                               L1_wt = l1, 
                                                               alpha = alph)
    
    return modelo
    

In [7]:
formula = "renda ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
C(tipo_residencia, Treatment('Casa')) + \
idade + \
tempo_emprego + \
qtd_filhos + \
qt_pessoas_residencia + 1"

lista_alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

for alp in lista_alphas:
    
    modelo_ridge = criar_modelo(data_frame = df_treino, formula_modelo = formula, l1 = 0.0001, alph = alp)
    
    print("*************************************************************************************************************")
    print(f"MODELO COM ALPHA = {alp}")
    print("*************************************************************************************************************")
    print(modelo_ridge.summary())


*************************************************************************************************************
MODELO COM ALPHA = 0
*************************************************************************************************************
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     159.6
Date:                Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                        11:53:57   Log-Likelihood:            -1.1550e+05
No. Observations:               11250   AIC:                         2.310e+05
Df Residuals:                   11225   BIC:                         2.312e+05
Df Model:                          25                                         
Covariance Type:            nonrobust           

*************************************************************************************************************
MODELO COM ALPHA = 0.005
*************************************************************************************************************
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     159.6
Date:                Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                        11:53:58   Log-Likelihood:            -1.1550e+05
No. Observations:               11250   AIC:                         2.310e+05
Df Residuals:                   11225   BIC:                         2.312e+05
Df Model:                          25                                         
Covariance Type:            nonrobust       

*************************************************************************************************************
MODELO COM ALPHA = 0.05
*************************************************************************************************************
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     159.6
Date:                Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                        11:54:00   Log-Likelihood:            -1.1550e+05
No. Observations:               11250   AIC:                         2.310e+05
Df Residuals:                   11225   BIC:                         2.312e+05
Df Model:                          25                                         
Covariance Type:            nonrobust        

#### Observações:

- Regularizando o modelo com ridge, não houve diferença em nenhum dos modelos, **R-squared**, **Adj. R-squared**, **F-statistic**, **AIC** e **BIC** permaneceram iguais em todos os modelos.



# 3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?

In [8]:
formula = "renda ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
C(tipo_residencia, Treatment('Casa')) + \
idade + \
tempo_emprego + \
qtd_filhos + \
qt_pessoas_residencia + 1"

lista_alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

for alp in lista_alphas:
    
    modelo_lasso = criar_modelo(data_frame = df_treino, formula_modelo = formula, l1 = 1, alph = alp)
    
    print("*************************************************************************************************************")
    print(f"MODELO COM ALPHA = {alp}")
    print("*************************************************************************************************************")
    print(modelo_lasso.summary())


*************************************************************************************************************
MODELO COM ALPHA = 0
*************************************************************************************************************
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     159.6
Date:                Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                        11:54:02   Log-Likelihood:            -1.1550e+05
No. Observations:               11250   AIC:                         2.310e+05
Df Residuals:                   11225   BIC:                         2.312e+05
Df Model:                          25                                         
Covariance Type:            nonrobust           

*************************************************************************************************************
MODELO COM ALPHA = 0.005
*************************************************************************************************************
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     166.3
Date:                Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                        11:54:03   Log-Likelihood:            -1.1550e+05
No. Observations:               11250   AIC:                         2.310e+05
Df Residuals:                   11226   BIC:                         2.312e+05
Df Model:                          24                                         
Covariance Type:            nonrobust       

*************************************************************************************************************
MODELO COM ALPHA = 0.05
*************************************************************************************************************
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.262
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     173.6
Date:                Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                        11:54:04   Log-Likelihood:            -1.1550e+05
No. Observations:               11250   AIC:                         2.310e+05
Df Residuals:                   11227   BIC:                         2.312e+05
Df Model:                          23                                         
Covariance Type:            nonrobust        

#### Observações:

- Regularizando o modelo com LASSO, houve diferença nos dois últimos modelos, com alpha 0.05 e 0.1.

- O último modelo aparentemente é o melhor dos analisados.

- Apesar de **R-squared** e **Adj. R-squared** não terem tido alterações, o **F-statistic** teve um aumento e a regularização retirou 3 variaveis do modelo, mandando o mesmo resultado em **R-squared** e **Adj. R-squared**.

- Como um modelo que atinge o mesmo resultado com menos variaveis é o mais adequado, o modelo com alpha = 0.1 foi o melhor.

- Entende-se que para valores de alpha muito próximos de 0, não tem muita diferença de treinar o modelo apenas com fit()


# 4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?

* Função para aplicar o stepwise

In [9]:
def aplicar_stepwise(modelo, X, y):
    
    # Converte X_design em um DataFrame
    X_df = pd.DataFrame(X, columns=X.design_info.column_names)
   
    while True:
        
        p_valores = pd.Series(modelo.pvalues[1:], index=X_df.columns[1:])

        # Verifico se o maior p-value é maior que 5%
        if p_valores.max() > 0.05:

            # Pego o nome dessa coluna com o maior p-valor
            coluna_para_remover = p_valores.idxmax()

            # Removo essa coluna do data_frame
            X_df.drop(coluna_para_remover, axis=1, inplace=True)

            # Treino outro modelo sem essa coluna
            modelo = sm.OLS(y, X_df).fit()


        else:
            break
            
    return modelo

* Criação do primeiro modelo

In [10]:
formula = "renda ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
C(tipo_residencia, Treatment('Casa')) + \
idade + \
tempo_emprego + \
qtd_filhos + \
qt_pessoas_residencia + 1"

y, X = patsy.dmatrices(data = df_treino, formula_like = formula)

modelo = sm.OLS(y, X).fit()

modelo.rsquared

0.26229033369455446

* Cria um modelo através do stepwise

In [11]:
modelo_stepwise = aplicar_stepwise(modelo, X, y)

print(modelo_stepwise.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     497.4
Date:                Tue, 25 Apr 2023   Prob (F-statistic):               0.00
Time:                        11:54:05   Log-Likelihood:            -1.1550e+05
No. Observations:               11250   AIC:                         2.310e+05
Df Residuals:                   11241   BIC:                         2.311e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

* Testo o modelo criado através do stepwise, com os dados de teste

In [12]:

# Gero a matriz de designer
y, X = patsy.dmatrices(data = df_teste, formula_like = formula)


# Converter a matriz de designer em um dataframe, pra poder remover as colunas que não são usadas no modelo ajustado
X_df = pd.DataFrame(X, columns=X.design_info.column_names)


# Pego só as colunas que serão usadas
X_df = X_df[ modelo_stepwise.model.exog_names ].copy()


# Testo o modelo
previsoes = modelo_stepwise.predict(X_df)


# Calculo o r-squared
r2_score(df_teste['renda'], previsoes)

0.2614951619659349

# 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

Primeiramente, vamos anaalisar os resultados obtidos:

1) Modelos com Regularização Ridge, o melhor dos modelos apresentou:

* R-squared:      26,2%

* Adj. R-squared: 26.1%

* AIC: 231000

* BIC: 231200

* Além disso, utilizou todas as variáveis geradas na Matriz de Designer, 24 variáveis e o intercepto.


2) Modelos com Regularização LASSO, o melhor dos modelos apresentou:

* R-squared:      26,2%

* Adj. R-squared: 26.1%

* AIC: 231000

* BIC: 231200

* Um diferencial, foi que 2 variáveis consideradas não significantes foram removidas do modelo, ficando com 22 variáveis e o intercepto.


3) Modelo criado com stepwise:

* R-squared:      26.1%

* Adj. R-squared: 26.1%

* AIC: 231000

* BIC: 231100

* Removeu muitas variáveis do modelo, ficando apenas com 8 variáveis e o intercepto.

Tendo em vista os dados acima, dos modelos criados, o melhor modelo foi o criado com o stepwise.

O $R^2$ teve uma queda de menos de 1% com relação ao demais modelos, foi o modelo que ficou mais simples e com menos variáveis.

Mesmo após gerar a Matriz de Designer que aumenta bastante o número de variáveis do modelo por conta das dummys geradas, ao aplicar o stepwise sobraram apenas 5 variáveis e o intercepto.

Além disso, nos testes com a base de teste, o $R^2$ ficou muito próximo do obtido nos treinos:
    
    Treino: 25.4%
    Teste: 25.3%

# 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

Primeiramente, vamos refazer o melhor modelo que obtivemos anteriormente utilizando o LASSO:

In [13]:
formula = "renda ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
C(tipo_residencia, Treatment('Casa')) + \
idade + \
tempo_emprego + \
qtd_filhos + \
qt_pessoas_residencia + \
1"

y_treino, x_treino = patsy.dmatrices(data = df_treino, formula_like = formula, return_type='dataframe')

modelo = sm.OLS(y_treino, x_treino).fit_regularized(method = 'elastic_net', 
                                                      refit = True, 
                                                      L1_wt = 1, 
                                                      alpha = 0.1)

r2_score(y_treino, modelo.predict(x_treino))

0.2622890514213765

Agora, verficiar o R2 com os dados de teste, primeiramente sem fazer nenhuma transformação:

In [14]:
y_teste, x_teste = patsy.dmatrices(data = df_teste, formula_like = formula, return_type='dataframe')

r2_score(y_teste, modelo.predict(x_teste))

0.2607337098980539

Transformando renda para o log:

In [15]:
formula = "np.log(renda) ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
C(tipo_residencia, Treatment('Casa')) + \
idade + \
tempo_emprego + \
qtd_filhos + \
qt_pessoas_residencia + \
1"

y_treino, x_treino = patsy.dmatrices(data = df_treino, formula_like = formula, return_type='dataframe')

modelo = sm.OLS(y_treino, x_treino).fit_regularized(method = 'elastic_net', 
                                                      refit = True, 
                                                      L1_wt = 1, 
                                                      alpha = 0.1)

r2_score(y_treino, modelo.predict(x_treino))

0.33050541125536315

In [16]:
y_teste, x_teste = patsy.dmatrices(data = df_teste, formula_like = formula, return_type='dataframe')

r2_score(y_teste, modelo.predict(x_teste))

0.3385554500203526

In [17]:
# Obteve-se um aumento de 7% no R2 dos dados de teste.

# 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

Greação das Matrizes de Designer

In [18]:
formula = "renda ~ \
sexo + \
posse_de_veiculo + \
posse_de_imovel + \
C(tipo_renda, Treatment('Assalariado')) + \
C(educacao, Treatment('Secundário')) + \
C(estado_civil, Treatment('Casado')) + \
C(tipo_residencia, Treatment('Casa')) + \
idade + \
tempo_emprego + \
qtd_filhos + \
qt_pessoas_residencia + 1"

# gero a Matriz de Designer para o treino
y_treino, x_treino = patsy.dmatrices(data = df_treino, formula_like = formula, return_type = 'dataframe')

y_teste, x_teste = patsy.dmatrices(data = df_teste, formula_like = formula, return_type = 'dataframe')

Geradação do modelo

In [19]:
# Gero um modelo basico de Árvore de regressão

modelo_arv = DecisionTreeRegressor(max_depth=5).fit(x_treino, y_treino)

#tree.plot_tree(decision_tree=modelo, filled=True, feature_names=x_treino.columns)

Teste e indicadores do modelo com os dados de **treino**

In [20]:
r2_treino = r2_score(y_treino, modelo_arv.predict(x_treino))

print(f"R2 para os dados de treino = {r2_treino}")

R2 para os dados de treino = 0.5174419082997118


Teste e indicadores do modelo com os dados de **teste**

In [21]:
r2_teste = r2_score(y_teste, modelo_arv.predict(x_teste))

print(f"R2 para os dados de treino = {r2_teste}")

R2 para os dados de treino = 0.43554838568446697


### Observações:

Com uma Árvore de Regressão com profundidade máxima = 5, foi possível obter um R2 melhor tanto para os dados de treino, quanto para os de teste.